### Práctica sobre Desarrollo de aplicaciones web con Bottle

Se desea ampliar la práctica 3 sobre los servicios web de la EMT creando una aplicación web que actúe a modo de capa de presentación. Para ello se va a utilizar Bottle.

Se pide crear una aplicación web que tenga las siguientes páginas:

*  Una página principal que mostrará un conjunto de varios enlaces que representan los servicios que ofrece la aplicación[1 punto]:

   * Servicio 1:Servicio de obtener la ruta optima entre dos destinos usando los servicios de la EMT
   
   * Servicio 2:Servicio de mostrar los lugares de interés entre dos destinos indicados.
   
   * Servicio 3:Servicio de mostrar las paradas de autobús más cercanas con toda la información acerca de la parada: líneas que pasan, frecuencia, ...
   
* Cuando el usuario pulsa sobre el servicio 1 se le mostrará un formulario en el que podrá introducir el nombre de la calle y número de un origen y un destino de la capital de Madrid, y cuando pulse sobre un botón de "Enviar", se le mostrará una nueva página que mostrará la descripción de la ruta. En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial. [3 puntos]

* Cuando el usuario pulsa sobre el servicio 2 se le mostrará un formulario en el que podrá introducir el nombre de la calle y número de un origen y un destino de la capital de Madrid, y cuando pulse sobre un botón de "Enviar", se le mostrará una nueva página que mostrará un listado con los lugares de interés recuperados en la ruta entre las dos calles. De cada resultado mostrará la información disponible: nombre, teléfono, calle,...En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial.[3 puntos]

* Este servicio es nuevo, y requiere el uso de otro servicio web de la EMT. En este caso, se va a utilizar el servicio GetStreet de GEO. Cuando el usuario pulsa sobre el servicio 3 se le mostrará un formulario en el que podrá introducir el nombre de una calle y un número de la capital de Madrid, y cuando pulse sobre un botón de "Enviar", se le mostrará una nueva página que mostrará la información de las paradas más cercanas a la calle con toda la información acerca de las líneas de autobuses que paran en esas paradas. En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial. [3 puntos]

## Normas de entrega

* Fecha tope de entrega: 22/11/2017
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre DesarrolloWeb_GrupoX donde X será el número de grupo correspondiente.

In [ ]:
from bottle import route, default_app, template, run, static_file, error, request
from lxml import etree
from xml.etree import ElementTree
from xml.etree.ElementTree import Element
import requests

#Devuelve una lista con las coordenadas x y, dada una calle y un número
def coordenadas(calle,numero):
    datos = {
        'idClient':'WEB.SERV.segonz09@ucm.es',
        'PassKey':'973F19F0-B74A-407C-82E1-37D96CC9940B',
        'description': calle,
        'streetNumber': numero,
        'Radius':'',
        'Stops':'',
        'statistics':'',
        'cultureInfo':''
    }

    url = 'https://servicios.emtmadrid.es:8443/geo/servicegeo.asmx/GetStreet'
    response = requests.post(url, data=datos)
    ficheroXML = open("infoCalle.xml", 'w')
    ficheroXML.write(response.text)
    ficheroXML.close()

    f= open("infoCalle.xml", "rt")
    
    #Comprobamos si se han encontrado la calle introducida
    #Si el servidor web nos devuelve un fichero XML que contenga más de una línea, la habrá encontrado
    #Si solo contiene una línea, no la ha econtrado
    numLineas=0
    for line in f:
        numLineas+=1
        if(numLineas > 1): #Si contiene más de una línea paramos el bucle porque ya sabemos que el resultado es bueno
            break
    if(numLineas < 2): #Si no tiene al menos 2 líneas, no ha encontrado la calle y salimos de la función devolviendo un -1
        return -1 #Esto lo hacemos con vistas a los ejercicios 4 y 5 para indicar que la calle no existe
    
    else: #Si el fichero tiene al menos 2 líneas, ha encontrado la calle y continuamos la ejecución
        arbol=ElementTree.parse(f)
        nodo = arbol.find('Site')
        coorX = nodo.find('CoordinateX')
        coorY = nodo.find('CoordinateY')
        xy=[]
        xy.append(coorX.text)
        xy.append(coorY.text)

        return xy

#A partir de las coordenadas de los dos destinos, vamos metiendo en la listaMostrar todo lo que vayamos a mostrar por pantalla.
def caminoOptimo(xyOrigen,xyDestino,listaMostrar):
    datos = {
        'idClient':'WEB.SERV.segonz09@ucm.es',
        'PassKey':'973F19F0-B74A-407C-82E1-37D96CC9940B',
        'statistics':'',
        'cultureInfo':'',
        'coordinateXFrom': xyOrigen[0],
        'coordinateYFrom':xyOrigen[1],
        'originName':'',
        'coordinateXTo': xyDestino[0],
        'coordinateYTo':xyDestino[1],
        'destinationName':'',
        'criteriaSelection':'13',
        'day':'',
        'month':'',
        'year':'',
        'hour':'',
        'minute':'',
        'GenerarAudio':''
    }

    url = 'https://servicios.emtmadrid.es:8443/servicemedia/servicemedia.asmx/GetStreetRoute'
    response = requests.post(url, data=datos)
    ficheroXML = open("infoRuta.xml", 'w')
    ficheroXML.write(response.text)
    ficheroXML.close()

    f= open("infoRuta.xml", "rt")
    arbol=ElementTree.parse(f)
    nodo = arbol.find('ListRouteData')
    nodo2= nodo.find('RouteData')
    nodo3=nodo2.find('DescriptionRouteData')

    fecha = nodo3.find('DescriptionDate')
    horaInicio = nodo3.find('DescriptionInitTime')
    horaFin = nodo3.find('DescriptionEstimateTimeArrival')
    trasbordos = nodo3.find('Transfers')
    duracion = nodo3.find('LongJourney')

    #quitamos lo que no valga en el texto
    fecha=fecha.text.replace("Fecha:","")
    horaInicio=horaInicio.text.replace("Hora de Salida:","")
    #NUEVO
    listaMostrar.append(fecha)
    listaMostrar.append(horaInicio)
    listaMostrar.append(horaFin.text)
    listaMostrar.append(trasbordos.text)
    listaMostrar.append(duracion.text)
   
    listaSecciones=nodo2.find('ListSectionRoute')
    #Recorro todas las secciones
    for seccion in listaSecciones.iter("Section"):
        secciones= list(seccion)
        for paso in secciones:
            if paso.tag == "WalkingLeg":
                leer= paso.find('SourceWalkingLeg')
                leer2=leer.find('RouteDescription')
                #si es lo ultimo que hace quitamos el " hasta"
                andar=leer2.text.replace(" hasta","")
                listaMostrar.append(andar)
            elif paso.tag == "BusLeg":
                leer= paso.find('SourceBusLeg')
                leer2=leer.find('RouteDescription')
                listaMostrar.append(leer2.text)

#dada la calle de origen y destino y sus números, llama a coordenadas para conseguir las coordenadas.
def origenDestino(cOrigen,nOrigen, cDestino,nDestino):
    xyOrigen=coordenadas(cOrigen,nOrigen)
    xyDestino=coordenadas(cDestino,nDestino)
    
    return xyOrigen,xyDestino
#Dadas las coordenadas va metiendo en la listaMostrar los mensajes que vamos a mostrar.             
def pOICaminoOptimo(xyOrigen,xyDestino,listaMostrar):
    datos = {
        'idClient':'WEB.SERV.segonz09@ucm.es',
        'PassKey':'973F19F0-B74A-407C-82E1-37D96CC9940B',
        'statistics':'',
        'cultureInfo':'',
        'coordinateXFrom': xyOrigen[0],
        'coordinateYFrom':xyOrigen[1],
        'originName':'',
        'coordinateXTo': xyDestino[0],
        'coordinateYTo':xyDestino[1],
        'destinationName':'',
        'criteriaSelection':'13',
        'day':'',
        'month':'',
        'year':'',
        'hour':'',
        'minute':'',
        'GenerarAudio':''
    }

    url = 'https://servicios.emtmadrid.es:8443/servicemedia/servicemedia.asmx/GetStreetRoute'
    response = requests.post(url, data=datos)
    ficheroXML = open("infoRuta.xml", 'w')
    ficheroXML.write(response.text)
    ficheroXML.close()

    f= open("infoRuta.xml", "rt")
    arbol=ElementTree.parse(f)
    f.close()
    
    nodo = arbol.find('POIS')
    listaPOIS=nodo.find('ListPOIS')
    
    for poi in listaPOIS.iter("POI"):
        nombre = poi.find('nombre')
        direccion = poi.find('direccion')
        nCalle = poi.find('numeroCalle')
        tlfn = poi.find('telefono')
        listaMostrar.append(nombre.text)
        listaMostrar.append(direccion.text)
        listaMostrar.append(nCalle.text)
        listaMostrar.append(tlfn.text)


#Paradas a apartir de una calle y un numero, va metiendo todo lo que va a mostrar en a lista
def encontrarParadas(calle,nCalle, listaMostrar):
    datos = {
        'idClient':'WEB.SERV.segonz09@ucm.es',
        'PassKey':'973F19F0-B74A-407C-82E1-37D96CC9940B',
        'description': calle,
        'streetNumber':nCalle,
        'Radius':'',
        'Stops':'',
        'statistics':'',
        'cultureInfo':''
    }

    url = 'https://servicios.emtmadrid.es:8443/geo/servicegeo.asmx/GetStreet'
    response = requests.post(url, data=datos)
    #lo escribimos en un fichero llamado infoCalle.xml
    ficheroXML = open("infoParadas.xml", 'w')
    ficheroXML.write(response.text)
    ficheroXML.close()
    #lo abrimos y nos sitiamos en la etiqueta Site
    try:
        f= open("infoParadas.xml", "rt")
    except:
        print ("No se ha podido abrir el archivo infoParadas.xml")
        exit()
        
    #lo convertimos en un arbol
    arbol=ElementTree.parse(f)
    sites = arbol.findall('Site')
    for site in sites:
        if site.find('Description').text.lower() == calle.lower() and str(site.find('StreetNumber').text) == str(nCalle):
            paradas = site.findall('Stop')
            for nodo in paradas:
                listaMostrar.append("Parada: ")
                listaMostrar.append(nodo.find('IdStop').text)
                listaMostrar.append(nodo.find('Name').text)
                lineas = nodo.findall('Line')
                for linea in lineas:
                    listaMostrar.append("Identificador linea: ")
                    listaMostrar.append(linea.find('IdLine').text)
                    listaMostrar.append("Etiqueta: ")
                    listaMostrar.append(linea.find('Label').text)
                    if (linea.find('HeaderA') != None):
                        listaMostrar.append("Cabecera A: ")
                        listaMostrar.append(linea.find('HeaderA').text)
                    if (linea.find('HeaderB') != None):
                        listaMostrar.append("Cabecera B: ")
                        listaMostrar.append(linea.find('HeaderB').text)
                    if (linea.find('Direction') != None):
                        listaMostrar.append("Direccion: ")
                        listaMostrar.append(linea.find('Direction').text)
                    if (linea.find('DayType') != None):
                        listaMostrar.append("Tipo de dia: ")
                        listaMostrar.append(linea.find('DayType').text)
                    if (linea.find('StartTime') != None):
                        listaMostrar.append("Hora de inicio: ")
                        listaMostrar.append(linea.find('StartTime').text)
                    if (linea.find('StopTime') != None):
                        listaMostrar.append("Hora de fin: ")
                        listaMostrar.append(linea.find('StopTime').text)
                    if (linea.find('MinimunFrequency') != None):
                        listaMostrar.append("Frecuencia minima: ")
                        listaMostrar.append(linea.find('MinimunFrequency').text)
                    if (linea.find('MaximumFrequency') != None):
                        listaMostrar.append("Frecuencia maxima: ")
                        listaMostrar.append(linea.find('MaximumFrequency').text)


#Pagina principal, con los 3 enlaces
@route('/')
def index():
    return template("index.tpl")
#Si pulsas en el primer enlace para coger las calles
@route('/cogerCalles')
def cogerCalles():
    return template("cogerCalles.tpl")
#Muestra la solucion del primer enlace
@route('/cogerCalles',method='POST') 
def do_cogerCalles():
    calleOrigen = request.forms.get('calleOrigen')
    ncalleOrigen = request.forms.get('ncalleOrigen')
    calleDestino = request.forms.get('calleDestino')
    ncalleDestino = request.forms.get('ncalleDestino')
    listaMostrar=[]#Aqui guardo todos los print que hacia en la practica 3 para enviarlo al template y que este los muestre
    listaMostrar.append(calleOrigen)
    listaMostrar.append(ncalleOrigen)
    listaMostrar.append(calleDestino)
    listaMostrar.append(ncalleDestino)
    #Cogemos las coordenadas
    xyOrigen,xyDestino = origenDestino(calleOrigen,ncalleOrigen,calleDestino,ncalleDestino)
    #Si no existe esas calles volvemos al formaulario para coger de nuevo las calles
    if (xyOrigen == -1 or xyDestino == -1):
        return template("errorCogerCalles.tpl")
    #si existe entonces consigo todos los mensajes que mostrare y los muestro.
    caminoOptimo(xyOrigen,xyDestino,listaMostrar)
    return template("rutaOptima.tpl",listaMostrar=listaMostrar)
#Para coger las calles del segundo enlace
@route('/puntosInteres')
def puntosInteres():
    return template("puntosInteres.tpl")
#Solucion del segundo enlace
@route('/puntosInteres',method='POST') 
def do_puntosInteres():
    calleOrigen = request.forms.get('calleOrigen')
    ncalleOrigen = request.forms.get('ncalleOrigen')
    calleDestino = request.forms.get('calleDestino')
    ncalleDestino = request.forms.get('ncalleDestino')
    listaMostrar=[]#Aqui guardo todos los print que hacia en la practica 3 para enviarlo al template y que este los muestre
    listaMostrar.append(calleOrigen)
    listaMostrar.append(ncalleOrigen)
    listaMostrar.append(calleDestino)
    listaMostrar.append(ncalleDestino)
    #Cogemos las coordenadas
    xyOrigen,xyDestino = origenDestino(calleOrigen,ncalleOrigen,calleDestino,ncalleDestino)
    #Si no existe esas calles volvemos al formaulario para coger de nuevo las calles
    if (xyOrigen == -1 or xyDestino == -1):
        return template("errorPuntosInteres.tpl")
    #si existe entonces consigo todos los mensajes que mostrare y los muestro.
    pOICaminoOptimo(xyOrigen,xyDestino,listaMostrar)
    return template("pois.tpl",listaMostrar=listaMostrar)
#Para coger la calle del tercer enlace
@route('/paradasCercanas')
def paradasCercanas():
    return template("paradasCercanas.tpl")
#Solcucion para mostrar el tercer enlace
@route('/paradasCercanas',method='POST') 
def do_paradasCercanas():
    calleOrigen = request.forms.get('calleOrigen')
    ncalleOrigen = request.forms.get('ncalleOrigen')
    
    listaMostrar=[]#Aqui guardo todos los print que hacia en la practica 3 para enviarlo al template y que este los muestre
    listaMostrar.append(calleOrigen)
    listaMostrar.append(ncalleOrigen)
    xyOrigen,xyDestino = origenDestino(calleOrigen,ncalleOrigen,"cascanueces",8)
    #Si no existe esas calles volvemos al formaulario para coger de nuevo las calles
    if (xyOrigen == -1):
        return template("errorParadasCercanas.tpl")
    #si existe entonces consigo todos los mensajes que mostrare y los muestro.
    encontrarParadas(calleOrigen,ncalleOrigen, listaMostrar)

    return template("mostrarParadas.tpl",listaMostrar=listaMostrar)
#Por si hay un error
@error(404)
def error404(error):
    return 'Error en la página solicitada'

run(host="localhost", port=8080, debug=True)